In [60]:
using MathOptInterface
using Ipopt
using Test
using LinearAlgebra
using DiffOpt
using DelimitedFiles

const MOI = MathOptInterface

MathOptInterface

In [61]:
nz = 10
nineq = 25
neq = 10

10

In [62]:
# read matrices from files
names = ["Q", "q", "G", "h", "A", "b"]
matrices = []

for name in names
    push!(matrices, readdlm(name*".txt", ' ', Float64, '\n'))
end
    
Q, q, G, h, A, b = matrices
q = vec(q)
h = vec(h)
b = vec(b);

In [63]:
optimizer = MOI.instantiate(Ipopt.Optimizer, with_bridge_type=Float64)

x = MOI.add_variables(optimizer, nz)

# define objective
quadratic_terms = MOI.ScalarQuadraticTerm{Float64}[]
for i in 1:nz
    for j in i:nz # indexes (i,j), (j,i) will be mirrored. specify only one kind
        push!(quadratic_terms, MOI.ScalarQuadraticTerm(Q[i,j],x[i],x[j]))
    end
end

objective_function = MOI.ScalarQuadraticFunction(MOI.ScalarAffineTerm.(q, x), quadratic_terms, 0.0)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), objective_function)
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MIN_SENSE)

# set constraints
for i in 1:nineq
    MOI.add_constraint(
        optimizer,
        MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(G[i,:], x), 0.),MOI.LessThan(h[i])
    )
end

for i in 1:neq
    MOI.add_constraint(
        optimizer,
        MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(A[i,:], x), 0.),MOI.EqualTo(b[i])
    )
end

In [64]:
dm = diff_model(optimizer)

#1 (generic function with 1 method)

In [65]:
z, λ, ν = dm.forward()

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      100
Number of nonzeros in inequality constraint Jacobian.:      250
Number of nonzeros in Lagrangian Hessian.............:       55

Total number of variables............................:       10
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       10
Total number of inequality constraints...............:       25
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       25

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

([-1.50146, -1.40531, 0.597702, -0.233848, 0.87574, -0.141102, 0.0414033, -0.995577, 0.738403, 0.588709], [-2.63697e-14, -4.59445e-14, 1.22901e-14, -1.97106e-15, 2.06348e-14, 2.37439e-15, 1.38352e-14, 8.92055e-15, 1.89919e-14, -1.58258e-15  …  9.60755e-15, -3.51949e-14, 3.80686e-14, 2.17293e-14, 7.96443e-15, 3.43789e-14, 1.95445e-14, 2.32037e-15, 1.45256e-15, 1.58126e-14], [-6.67454, 1.7444, 11.3345, 6.85921, -23.7225, -27.3077, 14.8993, -3.26951, -14.0398, -19.5364])

In [67]:
# obtain gradients
grads = dm.backward(names, ones(1,nz));  # using dl_dz=[1,1,1,1,1,....]

6-element Array{Any,1}:
 [-9.72469e-16 -2.17547e-16 … 1.15257e-16 1.82403e-17; -2.17547e-16 4.44671e-16 … -2.32758e-16 -2.54507e-16; … ; 1.15257e-16 -2.32758e-16 … 1.21833e-16 1.33355e-16; 1.82403e-17 -2.54507e-16 … 1.33355e-16 1.35198e-16]                                                               
 [6.4768e-16, -3.16422e-16, 7.24946e-17, 3.25893e-16, 3.64878e-16, -4.82649e-16, 2.61385e-16, 1.58521e-16, 1.64996e-16, 2.29652e-16]                                                                                                                                                   
 [-3.1051e-29 -5.33918e-29 … 2.80207e-29 2.49272e-29; -2.2629e-28 -2.54188e-28 … 1.33502e-28 1.10945e-28; … ; -2.73367e-30 -1.21844e-30 … 6.42049e-31 3.69385e-31; -3.50912e-29 -1.82548e-29 … 9.61177e-30 6.1119e-30]                                                                 
 [-3.20555e-29, -1.70532e-28, -6.34275e-29, 1.07289e-29, -4.3116e-29, 2.3768e-30, -2.18848e-29, -1.38529e-28, -1.69436e-29, -4.30571e-31

In [73]:
# read gradients from files
names = ["dQ", "dq", "dG", "dh", "dA", "db"]
grads_actual = []

for name in names
    push!(grads_actual, readdlm(name*".txt", ' ', Float64, '\n'))
end

grads_actual[2] = vec(grads_actual[2])
grads_actual[4] = vec(grads_actual[4])
grads_actual[6] = vec(grads_actual[6]);

In [79]:
# testing differences
atol=1e-2
rtol=1e-2

for i in 1:size(grads)[1]
    @test grads[i] ≈  grads_actual[i] atol=ATOL rtol=RTOL
end